<img src="data/images/lecture-notebook-header.png" />

# Data Stream Mining: Distinct Counting

Finding the number of unique users visiting Facebook or any other websites is a very common task as it represents an important metric to assess the popularity of a website. This task translates to a more general goal of finding the number of distinct elements in a set S. Counting the number of distinct elements in S becomes challenging when S and the expected number of distinct elements becomes very large, as straightforward solutions that calculate the exact count require large amounts of space beyond the main memory.

Counting distinct elements in a large data stream poses a significant challenge due to the following reasons:

* **Memory Limitations:** In a large data stream, it may not be feasible to store all the elements in memory. Traditional methods that require storing every element for counting distinct values become impractical when the data stream is too large to fit into memory. As a result, alternative approaches are needed to approximate the count without storing the entire stream.

* **Streaming Nature:** Data streams are typically continuous and unbounded, meaning that elements arrive one by one and cannot be revisited. This makes it impossible to perform multiple passes over the data stream to obtain an accurate count. Instead, the count must be estimated incrementally as new elements arrive.

* **Limited Processing Time:** In a streaming environment, there are constraints on processing time and real-time analysis. The counting algorithm should be efficient and provide approximate results in a timely manner. It is crucial to strike a balance between accuracy and computational complexity to ensure that the counting process can keep up with the stream velocity.

* **High Cardinality:** Large data streams often contain a high number of distinct elements, which exacerbates the memory and processing challenges. As the number of distinct elements increases, the memory required to store unique values or maintain a precise count grows, making it more difficult to achieve an accurate count within the given constraints.

To address these challenges, specialized algorithms and data structures have been developed for counting distinct elements in a data stream.
However, it is often not required to get the *exact* number of distinct elements, but a "good" estimate is enough. This is where approximation algorithms come into play. They trade-off speed vs. accuracy, where an important goal is to quantify and minimize the error of the estimated results. The Flajolet-Martin algorithm is such an approximation approach to count the number of distinct elements in large sets.

## Setting up the Notebook

### Make all Required Imports

In [ ]:
import numpy as np
import pandas as pd

import hashlib
import sys

from tqdm import tqdm

### Example Dataset

In the following, we use a publicly available [Apache access log provided by NASA](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html). To keep it simple and the file size small, the file contains only the IP address of each request.

In [ ]:
df_log = pd.read_csv('data/datasets/nasa-apache-logs/ip-only-nasa-access.log', header=None)

df_log.head()

In [ ]:
print('Number of log entries: {}'.format(df_log.shape[0]))
print('Number of unique IP addresses: {}'. format(len(set(df_log[0].to_list()))))

**Note:** Here we use a pandas frame mainly to display some samples and calculate the basic statistics. Below, we read the file again line by line since `df.iterrows()` to loop over all rows has a very poor performance.

---

## Naive Approach

If we have not constraints regarding memory, we can simply get the count of distinct elements in a stream using a set. A Python `set` uses a hashtable as its underlying data structure. Since looking up an item in a hashtable is an O(1) operation, on average, each membership check runs in O(1).

In [ ]:
S = set()

with open('data/datasets/nasa-apache-logs/ip-only-nasa-access.log') as file:
    for line in file:
        # Get the IP address
        ip = line.strip()
        # Add IP address to set
        S.add(ip)

print('Exact number of unique IP addresses: {}'.format(len(S)))

In [ ]:
size_in_bytes = sys.getsizeof(S)

print(size_in_bytes)

Of course, for our simple example dataset, we have no problem keeping the complete set in memory on any PC nowadays. However, it still allows for a comparison with the space requirements when using the Flajolet-Martin algorithm at the end.

---

## Flajolet-Martin Algorithm

The Flajolet-Martin algorithm is a popular and influential algorithm used for approximating the number of distinct elements in a large data stream. It was introduced by Philippe Flajolet and G. Nigel Martin in 1985. The algorithm utilizes the concept of bitwise counting and probabilistic hashing to estimate the distinct count. Here's a high-level overview of the Flajolet-Martin algorithm:

* Initialize a set of counters, each represented by a binary string. The length of the binary string determines the maximum number of leading zeros needed to be observed for each counter.

* As each element arrives in the data stream, hash it using a suitable hash function.

* For each hashed element, determine the number of trailing zeros in its binary representation. This information is used to update the corresponding counter. Specifically, if the number of trailing zeros is greater than the current value of the counter, the counter is updated with the new count of trailing zeros.

* Repeat steps 2 and 3 for a sufficiently large number of elements in the stream.

* Finally, estimate the distinct count based on the observed maximum count of trailing zeros across all counters. The estimated count is calculated using a formula based on statistical properties.

The Flajolet-Martin algorithm leverages the observation that the probability of encountering a specific number of trailing zeros in the binary representation of a hash value is proportional to 2 raised to the power of the number of trailing zeros. By analyzing the maximum count of trailing zeros across multiple counters, an estimate of the distinct count can be derived.

The accuracy of the Flajolet-Martin algorithm depends on factors such as the number of counters used and the length of the binary strings. By increasing these parameters, the algorithm can provide better approximations of the distinct count. However, there is a trade-off between accuracy and memory consumption. The Flajolet-Martin algorithm and its variants have been widely used for distinct count approximation in large-scale data processing, such as in distributed systems, stream processing, and data analytics.

While not immediately intuitive why it works (reasonably well?), the basic steps for the Flajolet-Martin algorithm are very straightforward. The key requirement is a hash function that maps each of the $n$ possible elements to at least $\log_2{n}$ bits. For example, since we know that there are only $2^{32}$ possible IPv4 addresses, we can choose a hash function $h(s)$ that maps elements $s$ to bit strings of length 32.

For each element $s$ in the stream, we then have to
* Calculate hash $h(s)$ to bit string of length $\log_2{n}$
* Calculate $r(s)$ as the number number of trailing 0s in $h(s)$
* Keep track of largest $r(s)$ in a variable $R$

Lastly, we return the estimate for the number of distinct elements as $2^R$. In the lecture, we discussed the probability of getting good estimates. In this notebook, we implement and test the Flajolet-Martin algorithm.


### Define Hash Function

The following function maps an arbitrary string to an integer:

* `num_digits`: specifies the maximum number of digits and therefore the range of the possible hash values. For example, if `num_digits=3` the possible values are in `[0, 999]`.

* `nr`: A simple way to simulate multiple hash functions. `nr` is just a factor the input string is multiple with to yield a different hash value. For example, `'Hello World'*3`, i.e., `nr=3`, results in `'Hello WorldHello WorldHello World'`

Note that the definition of "good" hash functions is a whole important subject on its own and beyond our scope here.


In [ ]:
def h(s: str, num_digits=1, nr=1):
    return int(hashlib.sha1((s*nr).encode("utf-8")).hexdigest(), 16) % (10 ** num_digits)

A quick test...

In [ ]:
s = 'Hello World'

print('Integer hash value of string "{}": {}'.format(s, h(s, num_digits=8)))

### Calculate Number of Trailing 0s

The method `get_r()` takes an integer as input and uses bitwise operations to calculate the number of trailing 0s.


In [ ]:
def get_r(x: int):
    return int(np.log2(x&-x))

Let's quickly test the method

In [ ]:
x = 4 # 4 in binary is 100

print('In binary, {} has {} trailing 0s.'.format(x, get_r(x)))

### Basic Algorithm (1 Hash Function)

We first consider only 1 hash function.

In [ ]:
R = 0

with open('data/datasets/nasa-apache-logs/ip-only-nasa-access.log') as file:
#with open('data/ip-only-sec-monthly-access.log') as file:
    for line in file:
        # Get IP adress
        ip = line.strip()
        # IP string => integer hash => uniform integer hash => number of trailing zeros
        r = get_r(h(ip, nr=1, num_digits=8))
        # Update maximum number of trailing zeros if needed
        if r > R:
            R = r

print('Maximum number of trailing zeros R: {}'.format(R))            
print('Estimated number of unique IP addresses: {}'. format(2**R))

With `nr=1` and `num_digits=8` we get a good estimate. In fact it's the best estimate since all estimates are a power of 2. So the closest alternatives would be 4,096 (for R=12) and 16,384 (for R=12), both are farther away from the true count of 7,637.

However, you can play with the parameters `nr` and `num_digits` to see that for some values R will be a value different from 13.

### Extended Algorithm (Multiple Hash Function)

Using $2^R$ as the estimate of the number of distinct numbers, the basic Flajolet-Martin algorithm suffers from an obvious problem. Firstly, all estimates are always a power of then. And secondly, if R is just off by one from the best value, this already doubles the estimate (R+1) or halves the estimate (R-1).

The way to address these issues is to use multiple hash functions and average the results. But again, given the basic estimate of $2^R$, requires some consideration when calculating the average. For example, using a simple mean would be biased towards larger estimates due to the exponential increase. While the median would be less biased, again the median would always be a power of 2 again.

The standard approach is to group into `num_groups` with each group containing `num_samples_per_group` elements. Each group is averaged using the median to get an unbiased average, and lastly the mean over all medians to calculate the final estimate. This naturally requires `num_samples_per_group * num_groups` hash functions.

In [ ]:
num_samples_per_group = 16
num_groups = 8
num_hashes = num_samples_per_group * num_groups

print('Running Flajolet-Martin with {} hash functions'.format(num_hashes))

#### Run Flajolet-Martin

Extending the basic algorithm to multiple hash functions is trivial. We only change from keeping track of one R value to `num_hashes` R values (e.g., using a list). Note that for the default values `num_samples_per_group = 16` and `num_groups = 8`, this might take a couple of minutes.

In [ ]:
num_lines = sum(1 for line in open('data/datasets/nasa-apache-logs/ip-only-nasa-access.log', 'r'))

Rs = np.array([0] * num_hashes)

with open('data/datasets/nasa-apache-logs/ip-only-nasa-access.log') as file:
    for line in tqdm(file, total=num_lines):
        # Get IP adress
        ip = line.strip()
        
        for i in range(num_hashes):
            # IP string => integer hash => uniform integer hash => number of trailing zeros
            r = int(get_r(h(ip, nr=i+1, num_digits=8)))
            # Update maximum number of trailing zeros if needed
            if r > Rs[i]:
                Rs[i] = r

print('All {} individual R values:'.format(num_hashes))
print(Rs)

#### Calculate all Estimates

With all R values, we can directly calculate all estimates $2^R$.

In [ ]:
estimates = np.power(2, Rs)

np.random.shuffle(estimates)

print('All {} individual estimates:'.format(num_hashes))
print(estimates)

Looking at the list of all estimates, we can see that some entries are very bad estimates. Of course, this has to be expected to do the randomized nature, and motivates the need for multiple hash functions and average the individual estimates.

#### Calculate Final Result by Averaging all Estimates

As discussed above, we calculate the final estimate as follows

* Split list of individual estimates into `num_groups` groups (each containing `num_samples_per_group` estimates)
* Calculate median of each group
* Calculate mean of all group medians

In [ ]:
medians = []

for split in np.split(estimates, num_groups):
    medians.append(np.median(split))
    
final_estimate = np.mean(medians)

print(final_estimate)

Depending on the values for `num_samples_per_group` and `num_groups`, the final estimate should be rather close to the true values of 7,168.0.

You will have noticed that increasing the number of hash functions significantly affects the overall runtime. Note that in practice this can be improved by optimized hash functions. However, optimal hash functions are beyond our scope here.

#### Check Size of Data Structure

Lastly, we can look at the space required -- the whole reason why we use approximate algorithms like Flajolet-Martin. Here, we simply estimate the required space by looking how many bytes are used to store the list of all values of R.

In [ ]:
size_in_bytes = sys.getsizeof(Rs)

print(size_in_bytes)

Unsurprisingly, the space required is always insignificant compared to storing set S. This difference will be even more pronounced if the number of distinct items increases. After all, we only used a rather small dataset in this notebook.

## Summary

The Flajolet-Martin algorithm is a powerful technique for estimating the number of distinct items in a large data stream. It offers a space-efficient approach that addresses the challenges of memory limitations and real-time processing. Here's a summary of the pros and cons of the Flajolet-Martin algorithm:

**Pros:**

* Memory Efficiency: The Flajolet-Martin algorithm requires a relatively small amount of memory compared to storing all the unique items in the data stream. It achieves this by utilizing a set of counters, each represented by a binary string, rather than explicitly storing the items themselves. This makes it suitable for scenarios where memory is limited.

* Real-Time Processing: The algorithm provides approximate distinct counts incrementally as the data stream is processed. It does not require multiple passes over the stream, enabling real-time analysis and immediate estimates of the distinct count. This is particularly valuable in streaming applications where timely insights are crucial.

* Scalability: The Flajolet-Martin algorithm scales well to large data streams with high cardinality. It can handle data streams containing millions or even billions of items, making it suitable for big data applications. By adjusting the number of counters and the length of the binary strings, the algorithm can achieve better accuracy as the stream size increases.

**Cons:**

* Approximate Results: The Flajolet-Martin algorithm provides an estimate of the distinct count, which may not be perfectly accurate. The accuracy depends on various factors, including the chosen parameters and the statistical properties of the data stream. While the estimates are usually close to the true count, there is a chance of error.

* Sensitivity to Distribution: The accuracy of the algorithm can be influenced by the distribution of distinct items in the data stream. Certain distributions, such as skewed or heavily concentrated data, may lead to less accurate estimations. The algorithm assumes a random distribution of trailing zeros, and deviations from randomness can affect the results.

* Parameter Tuning: The Flajolet-Martin algorithm requires choosing appropriate parameters, such as the number of counters and the length of the binary strings. These parameters impact the accuracy and memory usage of the algorithm. Selecting the optimal parameters may require some experimentation and understanding of the data characteristics.

In summary, the Flajolet-Martin algorithm is a valuable tool for estimating distinct counts in large data streams. It offers memory efficiency, real-time processing, and scalability. However, it provides approximate results, is sensitive to distribution, and requires careful parameter tuning. Despite its limitations, the algorithm has been widely adopted in various applications, including network traffic analysis, data stream mining, and approximate query processing.